# SOLID Principles

The SOLID principles is a set of guidelines to help with making code maintainable and extendable. 
* If you're doing research work and only have short snippets of code, it won't matter too much most of the time. 
* It's really helpful if you're building something larger or in a collaboration and want to maintain or extend the code in the future. In this case, following these guidelines can make your code more easily understood and modifiable by your collaborators, and open for extension in the future. 

In this notebook - we'll go through an example code to illustrate all of the principles. 

# Original Code

The code below shows a class called 'TimLab' that allows for management of Tim's lab. It does three main things:
* Adding students to the lab. 
* Finding out the total cost to fund the students based on their years needed and funding per year. 
* Taking care of the funding by identifying the funding source and associated account number. 

Running the code will output the total cost, the fund source, the fund account number and whether or not the students are actually funded. 

In [17]:
class TimLab:
    people = []
    funding_per_year = []
    years_needed = []
    status = 'unfunded'

    def __init__(self, year):
        self.year = year
    
    def add_student(self, name, funding, years):
        self.people.append(name)
        self.funding_per_year.append(funding)
        self.years_needed.append(years)

    def total_cost(self):
        total = sum(funding * years for funding, years in zip(self.funding_per_year, self.years_needed))
        return total

    def funding(self, fund, account_number):
        if fund == 'NSERC':
            print("NSERC funding.")
            print(f"Account number: {account_number}")
            self.status = "funded"
        elif fund == "CIHR":
            print("CIHR funding.")
            print(f"Account number: {account_number}")
            self.status = "funded"
        elif fund == "None":
            print("No funding available.")
            self.status == "not funded"
        else:
            raise Exception(f"Unknown funding source: {fund}")


aol2024 = TimLab(2024)
aol2024.add_student("Bing", 30, 1)
aol2024.add_student("Jesse", 30, 0)

print(aol2024.total_cost())
aol2024.funding("NSERC", 123456)
print(aol2024.status)

30
NSERC funding.
Account number: 123456
funded


As a quick note: this code looks okay - pretty tight, easy to understand, and does what we want. 
* As I mentioned before, for most of us during our research process, this is fine and you don't need to follow anything below. 
* But if you're looking to collaborate on your code and/or trying to write code that will lead to bigger pieces and need to be extended further down the road, you'd want to try to apply the following. 
* Additionally, it is often that you come back to code you haven't touched for a while to add new features or rewrite old features, and the following would help with that extension and understandability. 

# S - Single Responsibilty

The S in SOLID is for single responsibility. You ideally want your classes and methods to not do to much and have a single responsibility. This would allow for easy reuse in the future. In the original case, the class `TimLab()` has too many responsibilities. It has to add students (which makes sense), calculate the total funding needed by the lab (which makes sense), and figure out the funding information and process (which is too much). Additionally, the funding method is kind of messy, with a lot of if/else statements. We can rewrite the funding method into its own class.  

In [18]:
class TimLab:
    people = []
    funding_per_year = []
    years_needed = []
    status = 'unfunded'

    def __init__(self, year):
        self.year = year

    def add_student(self, name, funding, years):
        self.people.append(name)
        self.funding_per_year.append(funding)
        self.years_needed.append(years)

    def total_cost(self):
        total = sum(funding * years for funding, years in zip(self.funding_per_year, self.years_needed))
        return total
    
    # since we no long can directly change the status, we need a new method in TimLab that would allow us to change
    # the status of the TimLab object.
    def set_status(self, status):
        self.status = status

Making the funding function into its own class, we'll pass an instance of `TimLab()` into it to indicate which object it's looking at funding and its status. 

In [20]:
class FundingSources:
    def nserc(self, lab, account_number):
        print("NSERC funding.")
        print(f"Account number: {account_number}")
        lab.set_status("funded")

    def cihr(self, lab, account_number):
        print("CIHR funding.")
        print(f"Account number: {account_number}")
        lab.set_status("funded")

    def no_fund(self, lab):
        print("No funding")
        lab.set_status("Still not funded")


aol2024 = TimLab(2024)
aol2024.add_student("Bing", 30, 1)
aol2024.add_student("Jesse", 30, 0)

print(aol2024.total_cost())

funding = FundingSources()
funding.nserc(aol2024, 123456)

print(aol2024.status)
aol2024.funding_per_year.clear()

30
NSERC funding.
Account number: 123456
funded


As can be seen, there's no change in the output, but the code itself is much more usable as the responsibilties are appropriately divided and we can now change funding, for instance, without changing the entire lab object. 

# O - Open Closed

The O in SOLID is for open-closed. This means:
* Your classes should be open for extension - we should be able to add additional functionality easily
* While being closed for modification - code should be able to be extended, but not by modifying the original classes

In the above case with our new `FundingSources` class, we would have to modifying the funding class if we wanted to add a new source. So we can rewrite this using subclasses. 

In [28]:
from abc import ABC, abstractmethod

# in case you're unfamiliar - abstract method basically sets a blueprint that other subclasses must have
class FundingSources(ABC):
    @abstractmethod
    def fund(self, lab, account_number=None):
        pass

# each of the following is a subclass of FundingSources()
class Nserc(FundingSources):
    def fund(self, lab, account_number):
        print("NSERC funding.")
        print(f"Account number: {account_number}")
        lab.set_status("funded")

class Cihr(FundingSources):
    def fund(self, lab, account_number):
        print("CIHR funding.")
        print(f"Account number: {account_number}")
        lab.set_status("funded")

# now that we have subclasses, we can easily add a new funding source - SSHRC as a subclass, without changing any of the
# existing classes or subclasses
class Sshrc(FundingSources):
    def fund(self, lab, account_number):
        print("SSHRC funding.")
        print(f"Account name: {account_number}")
        lab.set_status("funded")

class NoFund(FundingSources):
    def fund(self, lab, account_number):
        print("No funding")
        lab.set_status("Still not funded")


aol2024 = TimLab(2024)
aol2024.add_student("Bing", 30, 1)
aol2024.add_student("Jesse", 30, 0)

print(aol2024.total_cost())

# change the funding to our new funding source
funding = Sshrc()
funding.fund(aol2024, 'SSHRCAcc')

print(aol2024.status)
aol2024.funding_per_year.clear()

30
SSHRC funding.
Account name: SSHRCAcc
funded


As can be seen, the we can now use our new funding source, SSHRC, very easily. And this allows us to add any number of new funding sources without changing the `FundingSources()` class. 

# L - Liskov Substitution Principle

The L is for Liskov Substitution Principle. This means that in our code, any objects we have should be able to be replaced by another subclass without breaking the program. 

If you look in the codeblock above, you'll notice that our new funding source, `SSHRC` doesn't take an account number - it needs an account name. We ran the above code and it still worked fine, but it's forcing the parameter to be what we want. Imagine if that parameter took the account number and did some other operation on it, which doesn't work when the input is a string. This would also be confusing code, as it doesn't show that SSHRC needs a different parameter. 

We can fix this by initializing the parameter. 

In [31]:
# now we no longer need to define `account_number` in our FundingSources() class
class FundingSources(ABC):
    @abstractmethod
    def fund(self, lab):
        pass

class Nserc(FundingSources):
    # for each of these classes, we can initalize it as a separate parameter - so here it would be an account number.    
    def __init__(self, account_number):
        self.account_number = account_number
    def fund(self, lab):
        print("NSERC funding.")
        print(f"Account number: {self.account_number}")
        lab.set_status("funded")

class Cihr(FundingSources):
    def __init__(self, account_number):
        self.account_number = account_number
    def fund(self, lab):
        print("CIHR funding.")
        print(f"Account number: {self.account_number}")
        lab.set_status("funded")

class Sshrc(FundingSources):
    # here we initalize it as account name
    def __init__(self, account_name):
        self.account_name = account_name
    def fund(self, lab):
        print("SSHRC funding.")
        print(f"Account name: {self.account_name}")
        lab.set_status("funded")

class NoFund(FundingSources):
    def fund(self, lab):
        print("No funding")
        lab.set_status("Still not funded")


aol2024 = TimLab(2024)
aol2024.add_student("Bing", 30, 1)
aol2024.add_student("Jesse", 30, 0)

print(aol2024.total_cost())

funding = Nserc("Tim2024_NSERC")
funding.fund(aol2024)

print(aol2024.status)
aol2024.funding_per_year.clear()

30
NSERC funding.
Account number: Tim2024_NSERC
funded


Again, the output doesn't change - but now we're not forcing the parameter `account_number` to do be multiple different things and it will still work if we replace an NSERC fund with SSHRC fund. 

# I - Interface Segregation

The I in SOLID is for interface segregation. It means that classes shouldn't need to depend on interfaces that they aren't using. In practice, this means that we should split large interfaces to smaller ones for usability. 

Suppose for SSHRC, Tim needs to get approval via email from the MIE Department before the funding can be processed. An intuitive way would be to add it as an abstract method in the original `FundingSources()` class as so. 

In [37]:
from abc import ABC, abstractmethod

class FundingSources(ABC):
    @abstractmethod
    def fund(self, lab):
        pass
    
    # new abstract method in our original class
    @abstractmethod
    def email_check(self, email):
        pass


class Nserc(FundingSources):   
    def __init__(self, account_number):
        self.account_number = account_number
    def fund(self, lab):
        print("NSERC funding.")
        print(f"Account number: {self.account_number}")
        lab.set_status("funded")

class Cihr(FundingSources):
    def __init__(self, account_number):
        self.account_number = account_number
    def fund(self, lab):
        print("CIHR funding.")
        print(f"Account number: {self.account_number}")
        lab.set_status("funded")

class Sshrc(FundingSources):
    def __init__(self, account_name):
        self.account_name = account_name
        # add a new approval variable to see if the funding was approved
        self.approved = False
    def fund(self, lab):
        print("SSHRC funding.")
        print(f"Account name: {self.account_name}")
        lab.set_status("funded")
        #add a new function to our NSERC class
    def email_check(self, email):
        print(f"Funding is approved by {email}")
        self.approved = True

class NoFund(FundingSources):
    def fund(self, lab):
        print("No funding")
        lab.set_status("Still not funded")


aol2024 = TimLab(2024)
aol2024.add_student("Bing", 30, 1)
aol2024.add_student("Jesse", 30, 0)

print(aol2024.total_cost())

funding = Nserc("Tim2024_NSERC")
funding.fund(aol2024)

print(aol2024.status)
aol2024.funding_per_year.clear()

30


TypeError: Can't instantiate abstract class Nserc with abstract methods email_check

But as can be seen, this would give an error as all subclasses would need to have this `email_check` function, even though not all of them needed (you can just pass an empty `email_check` function to the other subclasses to make it work). 

So we need to segregate this task from the original class and create a new class for it.

In [36]:
# new email approval class
class EmailApproval:
    approved = False

    def email_check(self, email):
        print(f"Funding is approved by {email}")
        self.approved = True

    def is_approved(self):
        return self.approved

class FundingSources(ABC):
    @abstractmethod
    def fund(self, lab):
        pass

class Nserc(FundingSources):

    def __init__(self, account_name):
        self.account_name = account_name
    def fund(self, lab):
        print("NSERC funding.")
        print(f"Account name: {self.account_name}")
        lab.set_status("funded")

class Cihr(FundingSources):
    def __init__(self, account_number):
        self.account_number = account_number

    def fund(self, lab):
        print("CIHR funding.")
        print(f"Account number: {self.account_number}")
        lab.set_status("funded")

class Sshrc(FundingSources):
    # here we initialize with an `approval` parameter, which is of our EmailApproval class
    def __init__(self, account_number, approval: EmailApproval):
        self.account_number = account_number
        self.approval = approval

    def fund(self, lab):
        if not self.approval.is_approved():
            raise Exception("Funding hasn't been approved via email.")
        print("SSHRC funding.")
        print(f"Account number: {self.account_number}")
        lab.set_status("funded")

class NoFund(FundingSources):
    def fund(self, lab):
        print("No funding")
        lab.set_status("Still not funded")


aol2024 = TimLab(2024)
aol2024.add_student("Bing", 30, 1)
aol2024.add_student("Jesse", 30, 0)

print(aol2024.total_cost())

# now we have an email approval instance to use for our SSHRC funding
e_approval = EmailApproval()
funding = Sshrc(123456, e_approval)
e_approval.email_check('mie@utoronto.ca')
funding.fund(aol2024)

print(aol2024.status)
aol2024.funding_per_year.clear()

30
Funding is approved by mie@utoronto.ca
SSHRC funding.
Account number: 123456
funded


In the output, we can see that now the funding is approved by MIE via email, and when we switch the funding source, we don't need to call this `e_approval`, still allowing the code to be flexible. 

# D - Dependency Inversion

Dependency inversion tries to ensure that we don't need to depending on specific instances of subclasses. In our code above, we initialize our `Sshrc()` subclass with the `approval: EmailApproval` parameter, meaning it'll only work with a specific class. But what if there are other types of approvals it could need in certain situations instead of email? 

To fix this, we make an abstract approval class to allow for us to pass any kind of approval into it.

In [39]:
# our new approval class
class Approval(ABC):
    @abstractmethod
    def is_approved(self):
        pass

# now our email approval is a subclass of Approval()
class EmailApproval(Approval):
    approved = False

    def email_check(self, email):
        print(f"Funding is approved by {email}")
        self.approved = True

    def is_approved(self):
        return self.approved

# we now have an additional commitee approval subclass which needs a committee name instead of email   
class CommitteeApproval(Approval):
    approved = False

    def committee_check(self, committee_name):
        print(f"Funding is approved by {committee_name}")
        self.approved = True

    def is_approved(self):
        return self.approved

class FundingSources(ABC):
    @abstractmethod
    def fund(self, lab):
        pass


class Nserc(FundingSources):

    def __init__(self, account_name):
        self.account_name = account_name

    def fund(self, lab):
        print("NSERC funding.")
        print(f"Account name: {self.account_name}")
        lab.set_status("funded")


class Cihr(FundingSources):
    def __init__(self, account_number):
        self.account_number = account_number

    def fund(self, lab):
        print("CIHR funding.")
        print(f"Account number: {self.account_number}")
        lab.set_status("funded")

# our Sshrc class now takes a general approval
class Sshrc(FundingSources):
    def __init__(self, account_number, approval: Approval):
        self.account_number = account_number
        self.approval = approval

    def fund(self, lab):
        if not self.approval.is_approved():
            raise Exception("Funding hasn't been approved.")
        print("SSHRC funding.")
        print(f"Account number: {self.account_number}")
        lab.set_status("funded")


class NoFund(FundingSources):
    def fund(self, lab):
        print("No funding")
        lab.set_status("Still not funded")


aol2024 = TimLab(2024)
aol2024.add_student("Bing", 30, 1)
aol2024.add_student("Jesse", 30, 0)

print(aol2024.total_cost())

# so now, if the situation calls for a committee approval, we can easily pass it into SSHRC
c_approval = CommitteeApproval()
funding = Sshrc(123456, c_approval)
c_approval.committee_check('Funds Committee')
funding.fund(aol2024)

print(aol2024.status)
aol2024.funding_per_year.clear()

30
Funding is approved by Funds Committee
SSHRC funding.
Account number: 123456
funded


Now its easy to change the type of approval the funding might need. 